In [62]:
import tensorflow as tf
import derive
reload(derive)

x = tf.placeholder(tf.float32, (2, 1))
b = tf.constant([3.0, 1.0], tf.float32, (1, 2))

fun = tf.matmul(b, x) ** 2
grad = derive.grad(fun, x)

point = [0.1, 0.3]

with tf.Session() as sess:
    print grad.eval({x: np.array(point).reshape(2, 1)})

[[ 3.60000014]
 [ 1.20000005]]


In [24]:
import derive
reload(derive)
import sympy as sm
import tensorflow as tf

fun = lambda x: (3 * x[0] + x[1]) ** 2
point = [0.1, 0.3]

grad = derive.grad(fun)
print grad(point)
hess = derive.hess(fun)
print hess(point)

x = tf.Variable(0, dtype = tf.float32)
y = tf.Variable(0, dtype = tf.float32)
fun = (3 * x + y) ** 2

grad = derive.grad(fun, [x, y])
hess = derive.hess(fun, [x, y])
with tf.Session() as sess:
    tf.initialize_all_variables()
    print grad.eval({x: point[0], y: point[1]})
    print hess.eval({x: point[0], y: point[1]})
    
x, y = sm.symbols('x, y')
fun = (3 * x + y) ** 2
grad = derive.grad(fun)
print grad(point)
hess = derive.hess(fun)
print hess(point)

x = T.dvector('x')
fun = (3 * x) ** 2
# grad = derive.grad(fun, x)
# theano.function([x], fun)(point)

[ 3.6       1.200006]
[[ 17.99994875   5.99995054]
 [  5.99995054   2.00008066]]
[ 3.60000014  1.20000005]
[[ 18.   6.]
 [  6.   2.]]
[3.5999999999999996, 1.2000000000000002]
[[18 18]
 [ 2  2]]


In [18]:
import theano
import theano.tensor as T

# http://deeplearning.net/software/theano/tutorial/gradients.html
    
x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(
    lambda i, gy, x: T.grad(gy[i], x),
    sequences = T.arange(gy.shape[0]),
    non_sequences = [gy, x]
)
f = theano.function([x], H, updates = updates)
print f([4, 4])

[[ 2.  0.]
 [ 0.  2.]]


In [76]:
import theano
import theano.tensor as T

# http://stackoverflow.com/questions/30500332/how-to-use-theano-gradient-hessian-example-needed

x = T.dvector('x')
y = T.dvector('y')
input = [x, y]
cost = T.sum(x ** 2 + y ** 2)

f = theano.gradient.hessian(cost, wrt = input)
h = theano.function(input, f)

x = [1, 2]
y = [1, 2]
print h(x, y)

[array([[ 2.,  0.],
       [ 0.,  2.]]), array([[ 2.,  0.],
       [ 0.,  2.]])]


In [3]:
import numpy as np
import scipy.optimize as opt
from converge import Converge
from numpy.linalg import inv

def minimize(f, x, g, H, epsilon = 1e-7, maxsteps = 100):
    x = np.array(x).astype(float)
    converge = Converge(f, epsilon, maxsteps)
    
    while not converge.done(x):
        d = -inv(H(x)).dot(g(x))
        x += d * opt.minimize(lambda eta: f(x + eta * d), 1).x
    return x

In [4]:
from scipy import stats
import scipy.optimize as opt
import derive

pdf = stats.multivariate_normal((23, 7)).pdf
fun = lambda x: -pdf(x)
x = [21, 6]
g = derive.grad(fun)
H = derive.hess(fun)
print minimize(fun, x, g, H) 
print opt.minimize(fun, x).x
print opt.minimize(fun, x, method = 'Newton-CG', jac = g, hess = H).x

[ 23.00000001   7.        ]
[ 23.           6.99999999]
[ 22.99999977   6.99999988]
